In [21]:
#db 연결 함수 .
#mydb = dbConnection() 이런식으로 사용하면 커넥션을 사용 할 수 있다.
def dbConnection():
    import pymysql as my
    mydb=my.connect(
    host="192.168.0.110",
    user="bigdata",
    passwd="123123",
    db="project"
    )
    return mydb

In [40]:
#기업 이름으로 db에 있는 peg 값을 가져옴
def searchCpPegInfo(name):
    mydb = dbConnection()
    mycursor=mydb.cursor()
    mycursor.execute("select peg from stock_data_table where cp_name = '{}'".format(name))
    myresult=mycursor.fetchall()
    return myresult


In [44]:
# test = searchCpPegInfo('삼성전자')
# print(test)
mydb = dbConnection()
mycursor=mydb.cursor()
mycursor.execute("select * from stock")
myresult=mycursor.fetchall()
myresult

(('000020', '동화약품'),
 ('000040', 'KR모터스'),
 ('000050', '경방'),
 ('000060', '메리츠화재'),
 ('000070', '삼양홀딩스'),
 ('000075', '삼양홀딩스우'),
 ('000080', '하이트진로'),
 ('000087', '하이트진로2우B'),
 ('000100', '유한양행'),
 ('000105', '유한양행우'),
 ('000120', 'CJ대한통운'),
 ('000140', '하이트진로홀딩스'),
 ('000145', '하이트진로홀딩스우'),
 ('000150', '두산'),
 ('000155', '두산우'),
 ('000157', '두산2우B'),
 ('000180', '성창기업지주'),
 ('000210', '대림산업'),
 ('000215', '대림산업우'),
 ('000220', '유유제약'),
 ('000225', '유유제약1우'),
 ('000227', '유유제약2우B'),
 ('000230', '일동홀딩스'),
 ('000240', '한국테크놀로지그룹'),
 ('000250', '삼천당제약'),
 ('000270', '기아차'),
 ('000300', '대유플러스'),
 ('000320', '노루홀딩스'),
 ('000325', '노루홀딩스우'),
 ('000370', '한화손해보험'),
 ('000390', '삼화페인트'),
 ('000400', '롯데손해보험'),
 ('000430', '대원강업'),
 ('000440', '중앙에너비스'),
 ('000480', '조선내화'),
 ('000490', '대동공업'),
 ('000500', '가온전선'),
 ('000520', '삼일제약'),
 ('000540', '흥국화재'),
 ('000545', '흥국화재우'),
 ('000547', '흥국화재2우B'),
 ('000590', 'CS홀딩스'),
 ('000640', '동아쏘시오홀딩스'),
 ('000650', '천일고속'),
 ('000660', 'SK하이닉스'),
 

In [41]:
#검색한 기업의 peg 값이 있으면 값을 출력해주고, 없으면 안내 텍스트를 출력시켜주기 위한 함수 
def searchCheck(cpresult):
    if len(cpresult) == 0:
        cpname = "PEG 값이 없습니다. 신규 상장주이거나 투자대상으로 적합하지 않습니다."
        return cpname
    
    sliceindex = str(cpresult).find(",")
    cpname = str(cpresult)[2:sliceindex]
    return cpname

#아래처럼 사용하면 된다. 웹에서 기업명을 받고, 받은 기업명으로 peg 값을 db에서 가져오고, 가져온 값이 peg값이 있으면 peg 값을 출력, 
#없으면 안내텍스트 출력되도록. 플라스크 파일이 있는 부분인 것을 확인
# @app.route("/main2") 
# def main2():
#     name1 = request.args.get("cpsearch")
#     cpresult = searchCpPegInfo(name1)
#     pegresult = searchCheck(cpresult)
#     return render_template("main.html",name=name1,peg = pegresult)


In [ ]:
# mydb = dbConnection()
# mycursor=mydb.cursor()
# mycursor.execute("select * from stoke")
# myresult=mycursor.fetchall()

In [46]:
# cpname = searchCheck(test)
# print(cpname)
# peg = searchCheck(test)
# print(peg)

0.682


In [3]:
#네이버 상장사 모든 데이터 크롤링 함수. 먼저 myresult 라는 변수를 통해 db의 기업코드,이름을 가져와야 한다.
#만약 db에 값이 없다면 http://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage 사이트에서 상장회사 엑셀 파일을 받은 뒤
#데이터 프레임으로 값을 읽어서 함수를 사용하도록 코드를 수정하던지, db에 값을 넣어서 사용하던지 하는 수정이 필요하다.

def datafunction():
    import requests
    from bs4 import BeautifulSoup
    import numpy as np
    import pandas as pd
    #db에서 값을 다 가져온다.
    mydb = dbConnection()
    mycursor=mydb.cursor()
    mycursor.execute("select * from stoke")
    myresult=mycursor.fetchall()
    
    mergedata = pd.DataFrame(columns=("매출액","영업이익","당기순이익","영업이익률","순이익률",
                                  "ROE(지배주주)","부채비율","당좌비율","유보율","EPS(원)",
                                  "BPS(원)","주당배당금(원)","시가배당률(%)","배당성향(%)"))
    #db 의 값 갯수 만큼 반복문 돌림 len(myresult)
    for i in range(0,len(myresult)):
        code = str(myresult[i])[2:8] #db에서 가져온 값을 슬라이싱 해서 종목 코드를 구함
        name = str(myresult[i])[10:-1] #종목 이름
        URL = "https://finance.naver.com/item/main.nhn?code={code}".format(code=code)#네이버 금융에서 기업 코드를 통해 재무제표 데이터를 가져온다
        html = requests.get(URL).text

        soup = BeautifulSoup(html, 'html.parser')

    #--------------per 크롤링 구간. None인 값은 konex 상장 주식이거나 per 데이터가 없는 종목. 자료얻기 어려우므로 걸러준다------------------
        per=soup.select_one('#_per')
        print('name : {} , per : {}'.format(name,per))
        if per == None:
            continue
#         else:
#             per=soup.select_one('#_per').string.replace(',','')
            
#         if float(per) < 0 or float(per) > 20:
#             continue
        
        
        
    #--------------per 크롤링 구간 끝------------------
        #beautifulsoup 로 해당 종목의 html 코드를 전부 긁어온다. 0 번째로 고정해서 저장하는건 리스트형태로 저장하되 값이 하나만 있어서.
        finance_html = soup.select('div.section.cop_analysis div.sub_section')[0]
        #긁어온 html 코드에서 thread th 부분만 선택하고, 양쪽 끝의 공백과 \n 부분은 삭제한 텍스트 데이터를 추출한다.
        th_data = [item.get_text().strip() for item in finance_html.select('thead th')]
        annual_date = th_data[3:7]  #잘라온 재무제표의 데이터중 왼쪽의 연간 날짜 부분. ex)['2017.06', '2018.06', '2018.12', '2019.12(E)']
        quarter_date = th_data[7:13] # 그 데이터의 오른쪽 분기별 날짜부분 ex)['2018.03', '2018.06', '2018.09', '2018.12', '2019.03', '2019.06(E)']
        
        #재무제표의 행 index
        finance_index = [item.get_text().strip() for item in finance_html.select('th.h_th2')][3:]
        #인덱스를 제외한 실질적인 수치들
        finance_data = [item.get_text().strip() for item in finance_html.select('td')]



        finance_data = np.array(finance_data) #데이터를 numpy 배열형태로 변환시킴
        finance_data.resize(len(finance_index), 10) #10개로 정렬시키도록... 재무제표 형태로 리사이징

        finance_date = annual_date + quarter_date #날짜부분을 붙여줌

        #데이터프레임에 인덱스와 열 데이터를 합산한다
        finance = pd.DataFrame(data=finance_data[0:,0:], index=finance_index, columns=finance_date)
        
        #합산 된 왼쪽 연간 재무제표 데이터
        annual_finance = finance.iloc[:, :4] 
        #오른쪽 분기별 재무제표 데이터
        #quarter_finance = finance.iloc[:, 4:]


        #exchangeList > li.on > a.head.usd > div > span.change
        #원하는 데이터 추출하기

        #annual_finance["2018.12"]
        #18년 12월 데이터만 잘라주고, eps 증가율 계산.
        #결산 연월이 다른 데이터같은 경우 일단 넘어감.
        
        #/////////////////////////////////////////////////////////
        #아래의 처리를 한 이유는 작년 재무제표 데이터와 제작년 재무제표 데이터만 잘라내서 eps 데이터를 구하는데, 종목마다 
        #결산 분기가 12월인 종목이 있고 1월, 3월 이런 종목들이 있어서 에러가 발생한다.
        #그래서 결산 분기가 12월이 아닌 종목들은 걸러내는 작업.
        #이 부분은 차후 수정해야 더 많은 데이터를 구할 수 있다. 
        #날짜 같은 경우 매개변수로 입력 받는 형식으로 수정해야 함
        #////////////////////////////////////////////////////////
        #작년 재무제표 데이터 부분만 잘라냄
        datetemp = finance.iloc[:, 2:3]
        temp = str(datetemp.columns) #형변환을 해서 데이터를 잘라내기 쉽게 변환
        colname1 = temp[8:-19] #결산 년도의 열 인덱스만 잘라내는 부분. 작년 인덱스ex)2018.12
        
        datetemp = finance.iloc[:,1:2]
        temp = str(datetemp.columns)
        colname2 = temp[8:-19]  #제작년 인덱스
        #연말의 12월이 아니면 걸러줌
        if colname1 != '2018.12':
            continue
        elif colname2 != '2017.12':
            continue
        else:
            data1= int(annual_finance.loc["EPS(원)","2018.12"].replace(',',''))#거를꺼 거르고 eps수치 구함
            data2 = int(annual_finance.loc["EPS(원)","2017.12"].replace(',',''))
        
        endPrice = perFunction(code)
        per = endPrice/data1
        #고 per 주는 섹터마다 다르긴 한데... 여기선 30으로 제한 
        if per < 0 or per > 30:
            continue
        #확정 데이터중 제작년 eps 데이터와 작년 eps를 계산하고 최종적으로 per 와 나눠 주가이익 성장배율인 peg 수치를 구한다.
        #보통 peg 는 1이하일 경우 투자가치가 있고, 1.5 이상이 되면 투자가치를 상실한다고들 한다.
        #  - 값은 의미가 없음. 거를 데이터
        peg=0
        if data1 - data2 < 0:    #이부분은 필요한 데이터만 받는다고 정해지면 거를꺼 다 거르고 할때 사용
            continue
        elif data1 - data2 == 0: #zerodivision 때문에. exception 처리해서 넘어가면
                                    #데이터가 안들어가고, 1정도는 더해도 미미한 수치이므로 이렇게 처리
            data1 = data1+1
        elif data2 == 0:
            data2 = data2+1
        else:
            peg = float(per)/ (((data1 - data2) / data2) *100) 
        #peg 수치 계산하는 공식이 사용되는 부분. 사실 정확한 peg 는 2년간의 데이터를 구하고자 하면 2년 평균 eps 수치를 기하평균을 사용하여 구하고 
        #각 해당 연도의 per 수치를 사용해서 구해야 하지만 공식 적용이 어려워 추후 수정. per 도 마찬가지. 
                                                    
        
        #위의 공식을 사용한 이후 나온 peg 가 0 이하인 기업은 의미가 없으므로 걸러줌
        if peg <= 0:  
            continue
        #걸른 이후 데이터를 반올림 시켜줌. 소수점 셋째 자리수까지..
        peg = round(float(per)/ (((data1 - data2) / data2) *100),3)
        
        #크롤링 데이터중 제일 최신데이터만 잘라냄.
        datetemp = annual_finance.iloc[:,2:3]
        datetemp2 = str(datetemp.columns)
        date = datetemp2[8:-19]
        
        #컬럼에 날짜 추가하기 위해 자르는 작업
        cdate = datetemp
        #크롤링 해온 기업이름에서 ' 부분을 자르기 위한 코드. name 에서 ' 찾고  int 형으로 변환, 앞뒤 번지수 찾아서 사용
        start= int(str(name).find("'"))+1
        end = int(str(name).rfind("'"))
        #print(ttest)
        #데이터프레임에 계산한 데이터들 병합 및 행열 전환(작업 편하게 하려고)
        qtest = np.transpose(cdate)
        qtest["cp_code"] = code
        qtest["cp_name"] = name[start:end]
        qtest["date"] = date
        qtest["per"] = round(per,2)
        qtest["peg"] = peg

        mergedata = pd.concat([mergedata,qtest])
        print("count : {}, name : {}".format(i,name))
      #  print(name,round(peg,2))
    return mergedata
#------------------------------------------------for문 종료----------------------------------------------


In [62]:
cpdata = datafunction()
cpdata

name :  '비츠로시스' , per : <em id="_per">-0.98</em>
name :  '팬스타엔터프라이즈' , per : <em id="_per">-44.00</em>
name :  '피앤텔' , per : <em id="_per">-0.97</em>
name :  '케이피티유' , per : <em id="_per">10.96</em>
name :  '텔레칩스' , per : <em id="_per">20.94</em>


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:148: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



count : 4, name :  '텔레칩스'
name :  '삼영엠텍' , per : <em id="_per">33.01</em>
name :  'APS홀딩스' , per : <em id="_per">90.00</em>
name :  '에이디칩스' , per : <em id="_per">-7.55</em>
name :  '대한뉴팜' , per : <em id="_per">73.68</em>
name :  '키이스트' , per : <em id="_per">-35.05</em>
name :  '아이디스홀딩스' , per : <em id="_per">6.45</em>
count : 10, name :  '아이디스홀딩스'
name :  '한컴시큐어' , per : <em id="_per">161.75</em>
name :  '유신' , per : <em id="_per">-9.77</em>
name :  '엑사이엔씨' , per : <em id="_per">-14.17</em>
name :  '제이브이엠' , per : <em id="_per">32.12</em>
name :  '테이팩스' , per : <em id="_per">20.44</em>
name :  '신한지주' , per : <em id="_per">6.74</em>
count : 16, name :  '신한지주'
name :  '신스타임즈' , per : <em id="_per">-31.94</em>
name :  '유진로봇' , per : <em id="_per">-129.00</em>
name :  '유앤아이' , per : <em id="_per">-4.72</em>
name :  '에스에프에이' , per : <em id="_per">8.91</em>
name :  '코위버' , per : <em id="_per">14.41</em>
name :  '신화인터텍' , per : <em id="_per">-34.15</em>
name :  '포스링크' , per : <em id="_per">-3

count : 158, name :  'DMS'
name :  '디지털대성' , per : <em id="_per">36.23</em>
name :  '아이씨케이' , per : <em id="_per">-26.55</em>
name :  '웹젠' , per : <em id="_per">11.47</em>
count : 161, name :  '웹젠'
name :  '코스온' , per : <em id="_per">-79.15</em>
name :  '누리플랜' , per : <em id="_per">54.77</em>
name :  '휴켐스' , per : <em id="_per">8.37</em>
name :  '유아이디' , per : <em id="_per">-2.79</em>
name :  '엔텔스' , per : <em id="_per">50.59</em>
name :  '대호에이엘' , per : <em id="_per">174.46</em>
name :  'KODEX200' , per : None
name :  '에스텍' , per : <em id="_per">7.30</em>
count : 169, name :  '에스텍'
name :  '라이트론' , per : <em id="_per">-3.86</em>
name :  '대웅제약' , per : <em id="_per">-118.12</em>
name :  '한세엠케이' , per : <em id="_per">18.61</em>
name :  'KOSEF200' , per : None
name :  'DSR제강' , per : <em id="_per">10.06</em>
count : 174, name :  'DSR제강'
name :  '아이에스이커머스' , per : <em id="_per">-8.04</em>
name :  '현대백화점' , per : <em id="_per">8.24</em>
name :  '한솔시큐어' , per : <em id="_per">-6.88</em>
name

name :  '중앙에너비스' , per : <em id="_per">53.08</em>
name :  '조선내화' , per : <em id="_per">6.73</em>
count : 310, name :  '조선내화'
name :  '대동공업' , per : <em id="_per">115.58</em>
name :  '가온전선' , per : <em id="_per">836.96</em>
name :  '삼일제약' , per : <em id="_per">-19.89</em>
name :  '흥국화재' , per : <em id="_per">5.91</em>
name :  '흥국화재우' , per : <em id="_per">7.61</em>
name :  '흥국화재2우B' , per : <em id="_per">30.43</em>
name :  'CS홀딩스' , per : <em id="_per">11.76</em>
count : 317, name :  'CS홀딩스'
name :  '동아쏘시오홀딩스' , per : <em id="_per">-3.67</em>
name :  '천일고속' , per : <em id="_per">-514.48</em>
name :  'SK하이닉스' , per : <em id="_per">3.09</em>
count : 320, name :  'SK하이닉스'
name :  '영풍' , per : <em id="_per">12.16</em>
name :  'LS네트웍스' , per : <em id="_per">16.16</em>
name :  '유수홀딩스' , per : <em id="_per">23.27</em>
name :  '현대건설' , per : <em id="_per">15.77</em>
count : 324, name :  '현대건설'
name :  '현대건설우' , per : <em id="_per">52.28</em>
name :  '이화산업' , per : <em id="_per">-45.44</em>
name

...

name :  '에치에프알' , per : <em id="_per">-288.51</em>
name :  '에코마케팅' , per : <em id="_per">38.57</em>
count : 2409, name :  '에코마케팅'
name :  '자비스' , per : None
name :  'KOSEF미국달러선물인버스2X(합성)' , per : None
name :  '솔트웍스' , per : <em id="_per">4,010.00</em>
name :  'TIGER코스닥150' , per : None
name :  '와이아이케이' , per : <em id="_per">5.76</em>
count : 2414, name :  '와이아이케이'
name :  'SK3호스팩' , per : <em id="_per">116.94</em>
name :  '이엠티' , per : None
name :  'KINDEX골드선물인버스2X(합성H)' , per : None
name :  '라온테크' , per : None
name :  '시큐센' , per : None
name :  'TIGER코스닥150레버리지' , per : None
name :  '메디안디노스틱' , per : None
name :  'KODEX코스닥150레버리지' , per : None
name :  '질경이' , per : None
name :  '에이원알폼' , per : None
name :  'JW생명과학' , per : <em id="_per">24.57</em>
name :  '세원' , per : <em id="_per">43.97</em>
name :  '에스트래픽' , per : <em id="_per">-12.74</em>
name :  'KBSTARV&S셀렉트밸류' , per : None
name :  '자이글' , per : <em id="_per">-19.09</em>
name :  '씨알푸드' , per : None
name :  '슈프리마' , per : <em id="

name :  '씨앤지하이테크' , per : <em id="_per">5.70</em>
name :  'IBKS제6호스팩' , per : <em id="_per">258.89</em>
name :  '크라운제과' , per : <em id="_per">9.46</em>
name :  '미래에셋대우스팩1호' , per : <em id="_per">273.75</em>
name :  'AP시스템' , per : <em id="_per">15.57</em>
count : 2592, name :  'AP시스템'
name :  '영화테크' , per : <em id="_per">14.75</em>
name :  'KINDEX러시아MSCI(합성)' , per : None
name :  '한화수성스팩' , per : <em id="_per">111.58</em>
name :  'TIGER지속배당' , per : None
name :  'KBSTAR고배당' , per : None
name :  '앤유엔터테인먼트' , per : None
name :  'KODEX미디어&엔터테인먼트' , per : None
name :  'KODEXIT' , per : None
name :  'KODEX경기소비재' , per : None
name :  'KODEX필수소비재' , per : None
name :  'KODEX헬스케어' , per : None
name :  'ARIRANG중형주저변동50' , per : None
name :  '파워풀엑스' , per : None
name :  '명진홀딩스' , per : None
name :  '현대중공업지주' , per : <em id="_per">19.31</em>
name :  '현대일렉트릭' , per : <em id="_per">-2.18</em>
name :  '현대건설기계' , per : <em id="_per">9.24</em>
count : 2609, name :  '현대건설기계'
name :  '경동도시가스' , per : <e

name :  '화신테크' , per : <em id="_per">-7.25</em>
name :  '현대글로비스' , per : <em id="_per">14.02</em>
name :  '유니테스트' , per : <em id="_per">4.60</em>
count : 2773, name :  '유니테스트'
name :  '동국제약' , per : <em id="_per">11.57</em>
count : 2774, name :  '동국제약'
name :  '에스엔피제네틱스' , per : None
name :  '에코프로' , per : <em id="_per">14.88</em>
name :  '비엠티' , per : <em id="_per">7.71</em>
count : 2777, name :  '비엠티'
name :  '하나금융지주' , per : <em id="_per">4.91</em>
count : 2778, name :  '하나금융지주'
name :  '바이오솔루션' , per : <em id="_per">190.95</em>
name :  '이수앱지스' , per : <em id="_per">-14.30</em>
name :  '메디톡스' , per : <em id="_per">36.66</em>
name :  '한컴MDS' , per : <em id="_per">20.57</em>
name :  '쇼박스' , per : <em id="_per">47.64</em>
name :  '펩트론' , per : <em id="_per">-73.06</em>
name :  '모바일어플라이언스' , per : <em id="_per">52.28</em>
name :  '픽셀플러스' , per : <em id="_per">-5.06</em>
name :  '네패스신소재' , per : <em id="_per">-26.22</em>
name :  '동아엘텍' , per : <em id="_per">6.45</em>
count : 2788, name :

name :  '비상교육' , per : <em id="_per">50.05</em>
name :  '진양홀딩스' , per : <em id="_per">138.42</em>
name :  '머큐리' , per : <em id="_per">10.75</em>
count : 2928, name :  '머큐리'
name :  '서암기계공업' , per : <em id="_per">21.54</em>
count : 2929, name :  '서암기계공업'
name :  '세운메디칼' , per : <em id="_per">16.38</em>
count : 2930, name :  '세운메디칼'
name :  '미래에셋벤처투자' , per : <em id="_per">8.38</em>
count : 2931, name :  '미래에셋벤처투자'
name :  'S&TC' , per : <em id="_per">58.44</em>
name :  'KOSEFKRX100' , per : None
name :  '상상인인더스트리' , per : <em id="_per">-0.07</em>
name :  'SBS미디어홀딩스' , per : <em id="_per">28.77</em>
count : 2935, name :  'SBS미디어홀딩스'
name :  '비티원' , per : <em id="_per">67.07</em>
name :  '월덱스' , per : <em id="_per">6.91</em>
count : 2937, name :  '월덱스'
name :  '우림기계' , per : <em id="_per">11.76</em>
name :  '씨큐브' , per : <em id="_per">19.23</em>
count : 2939, name :  '씨큐브'
name :  'KODEX일본TOPIX100' , per : None
name :  '모베이스' , per : <em id="_per">8.49</em>
count : 2941, name :  '모베이스'
na

,BPS(원),EPS(원),ROE(지배주주),cp_code,cp_name,date,peg,per,당기순이익,당좌비율,매출액,배당성향(%),부채비율,순이익률,시가배당률(%),영업이익,영업이익률,유보율,주당배당금(원)
2018.12,"7,763",683,10.30,054450,텔레칩스,2018.12,0.025,12.59,92,220.03,"1,261",16.27,42.21,7.31,1.40,82,6.47,"1,428.16",120
2018.12,"23,839","2,178",10.72,054800,아이디스홀딩스,2018.12,0.067,6.24,538,233.51,"5,563",8.22,30.90,9.67,1.47,548,9.84,"4,462.11",200
2018.12,"75,923","6,657",9.21,055550,신한지주,2018.12,0.729,5.95,"31,983",,"238,119",23.86,"1,153.98",13.43,4.04,"44,994",18.90,"1,291.55","1,600"
2018.12,"144,125","13,881",10.32,057050,현대홈쇼핑,2018.12,0.198,7.13,"1,666",98.98,"10,177",13.33,51.24,16.37,1.92,"1,123",11.04,"2,771.35","1,900"
2018.12,"17,357","3,191",19.82,058470,리노공업,2018.12,0.719,14.74,486,902.45,"1,504",34.25,7.52,32.35,2.34,512,34.05,"3,397.27","1,100"
2018.12,"1,659",139,,058630,엠게임,2018.12,14.636,21.37,25,211.53,271,,27.29,9.34,,36,13.35,247.98,
2018.12,"3,737",265,7.30,058850,KTcs,2018.12,0.160,9.38,114,151.67,"10,161",33.22,116.60,1.12,3.62,125,1.23,567.45,90
2018.12,"2,766",444,18.67,059090,미코,2018.12,0.213,10.54,323,197.80,"2,274",8.86,123.79,14.21,0.85,454,19.95,605.31,40
2018.12,"4,983",421,8.83,060150,인선이엔티,2018.12,0.037,15.01,155,51.16,"1,760",-,68.84,8.80,,239,13.56,894.02,
2018.12,"5,237",751,15.47,060250,NHN한국사이버결제,2018.12,0.270,15.51,168,131.98,"4,327",18.55,141.33,3.88,1.23,218,5.03,"1,093.50",143


In [2]:
#단일 종목 데이터 크롤링 함수 신규상장주들이 추가될 시 해당 함수를 사용한다. 종목의 이름과 코드를 입력하여 사용한다.
#위 함수와 코드가 거의 유사하므로 주석은 스킵..
def singleDataFunction(code,name):
    import requests
    from bs4 import BeautifulSoup
    import numpy as np
    import pandas as pd
    mergedata = pd.DataFrame(columns=("매출액","영업이익","당기순이익","영업이익률","순이익률",
                                  "ROE(지배주주)","부채비율","당좌비율","유보율","EPS(원)",
                                  "BPS(원)","주당배당금(원)","시가배당률(%)","배당성향(%)"))
    cpcode = code
    cpname = name
    URL = "https://finance.naver.com/item/main.nhn?code={cpcode}".format(cpcode=cpcode)
    html = requests.get(URL).text

    soup = BeautifulSoup(html, 'html.parser')

#--------------per 크롤링 구간 None인 값은 konex 상장 주식이거나 per 데이터가 없는 종목. ------------------
    per=soup.select_one('#_per')
    print('name : {} , per : {}'.format(cpname,per))
    if per == None:
        print("per 데이터가 없습니다. 데이터를 확인하여 주십시오")
        return
    else:
        per=soup.select_one('#_per').string.replace(',','')

    if float(per) < 0 or float(per) > 20:
        print("per 데이터가 0 미만입니다.")
        return


#--------------per 크롤링 구간 끝------------------

    html = requests.get(URL).text

    soup = BeautifulSoup(html, 'html.parser')

    finance_html = soup.select('div.section.cop_analysis div.sub_section')[0]

    th_data = [item.get_text().strip() for item in finance_html.select('thead th')]
    annual_date = th_data[3:7]
    quarter_date = th_data[7:13]

    finance_index = [item.get_text().strip() for item in finance_html.select('th.h_th2')][3:]

    finance_data = [item.get_text().strip() for item in finance_html.select('td')]



    finance_data = np.array(finance_data)
    finance_data.resize(len(finance_index), 10)

    finance_date = annual_date + quarter_date


    finance = pd.DataFrame(data=finance_data[0:,0:], index=finance_index, columns=finance_date)

    annual_finance = finance.iloc[:, :4]
   
  
    datetemp = finance.iloc[:, 2:3]
    temp = str(datetemp.columns)
    colname1 = temp[8:-19]

    datetemp = finance.iloc[:,1:2]
    temp = str(datetemp.columns)
    colname2 = temp[8:-19]

    if colname1 != '2018.12':
        print("18년 12월 날짜의 데이터가 없습니다")
        return
    elif colname2 != '2017.12':
        print("해당날짜의 데이터가 없습니다")
        return
    else:
        data1= int(annual_finance.loc["EPS(원)","2018.12"].replace(',',''))
        data2 = int(annual_finance.loc["EPS(원)","2017.12"].replace(',',''))


    #print(ttest)


    #확정 데이터중 제작년 eps 데이터와 작년 eps를 계산하고 최종적으로 per 와 나눠 주가이익 성장배율인 peg 수치를 구한다.
    #보통 peg 는 1이하일 경우 투자가치가 있고, 1.5 이상이 되면 투자가치를 상실한다고들 한다.
    #  - 값은 의미가 없음. 거를 데이터
    peg=0
    if data1 - data2 < 0:    #이부분은 필요한 데이터만 받는다고 정해지면 거를꺼 다 거르고 할때 사용
        print("eps증가율이 마이너스 값입니다.")
        return
    elif data1 - data2 == 0: #zerodivision 때문에. exception 처리해서 넘어가면
                                #데이터가 안들어가고, 1정도는 더해도 미미한 수치이므로 이렇게 처리
        data1 = data1+1
    else:
        peg = float(per)/ (((data1 - data2) / data2) *100)

    if peg <= 0:
        print("peg 수치가 0 이하입니다.")
        return
    peg = round(float(per)/ (((data1 - data2) / data2) *100),3)


    datetemp = annual_finance.iloc[:,2:3]
    datetemp2 = str(datetemp.columns)
    date = datetemp2[8:-19]

    ttest = datetemp

    #print(ttest)
    #데이터프레임에 계산한 데이터들 병합 및 행열 전환(작업 편하게 하려고)
    qtest = np.transpose(ttest)
    qtest["code"] = cpcode
    qtest["name"] = cpname
    qtest["date"] = date
    qtest["per"] = per
    qtest["peg"] = peg


  #  print(name,round(peg,2))
    return qtest

In [65]:

#데이터 엑셀화 함수. 매개변수는 엑셀파일로 만들 합산된 데이터프레임
def excelWrite(cpdata):
    import openpyxl
    #엑셀에 값 저장하기 전에 컬럼 이름을 변경시켜준다. db에 저장하기 편하게 하기 위한 작업
    cpdata = cpdata.rename(columns = {'매출액': 'total_revenue','당기순이익': 'total_income','영업이익': 'gross_profit','당좌비율': 'quick_ratio',
                             'EPS(원)': 'eps','BPS(원)': 'bps','ROE(지배주주)': 'roe','배당성향(%)': 'payout_ratio',
                             '부채비율': 'debt_ratio','순이익률': 'net_profit_margin','시가배당률(%)': 'yield_ratio','영업이익률': 'orperating_margin',
                             '유보율': 'reserve_ration','주당배당금(원)': 'per_share'})

    #병합된 데이터를 엑셀문서로 만들어준다. 데이터프레임의 컬럼과 엑셀문서에 생성하는 컬럼명이 일치해야 한다.
    #파일명, 시트명, 컬럼명 확인 잘 하고 실행. 
    cpdata.to_excel('./stock_data3.xlsx',
                    sheet_name='Sheet1',
                    columns=("cp_code","cp_name","date","peg","per","total_revenue","total_income","gross_profit","quick_ratio","eps","bps","roe",
                             "payout_ratio","debt_ratio","net_profit_margin","yield_ratio",
                             "orperating_margin","reserve_ration","per_share"),
                    header=True,
                    index=0,
                    index_label=None,
                    startrow=0,
                    startcol=0,
                    engine=None,
                    merge_cells=True,
                    encoding="euc_kr",
                    inf_rep='inf',
                    verbose=True,
                    freeze_panes=None)


In [66]:
excelWrite(cpdata)


In [46]:
#db에 엑셀 데이터 입력하는 함수
def dbInsertFunction():
    from openpyxl import Workbook
    from openpyxl import load_workbook
    mydb = dbConnection()
    mycursor=mydb.cursor()
    #IGNORE 키워드는 Mysql 5.6 이상 버전에서 데이터에 null 이 아닌 '' 값이 (빈값) 들어갈경우 에러가 나기 때문에 붙여주었다. 
    sql = 'insert IGNORE into stock_data_table values(%s, %s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
    #엑셀파일로딩. data_only 옵션은 수식을 가지고 있을시 수식을 포함할지 데이터만 포함할지 여부 결정.
    wb = load_workbook('stock_data3.xlsx',data_only=True)
    ws = wb['Sheet1']
    #iter() 함수를 통해 열의 데이터를 쭉 뽑아낸다. 자바의 이터레이터 같은 기능
    #mysql 버전 올라가면서 commit 해줘야 db에 값이 들어간다. 확인하자..
    iter_rows = iter(ws.rows)
    next(iter_rows)
    for row in iter_rows:
        mycursor.execute(sql, (row[0].value, row[1].value, row[2].value, row[3].value, row[4].value, row[5].value, row[6].value
                              , row[7].value, row[8].value, row[9].value, row[10].value, row[11].value, row[12].value, row[13].value
                              , row[14].value, row[15].value, row[16].value, row[17].value, row[18].value))
        mydb.commit()

In [47]:
#dbInsertFunction()

054450 텔레칩스 2018.12 0.025 12.59 1,261 92
054800 아이디스홀딩스 2018.12 0.067 6.24 5,563 538
055550 신한지주 2018.12 0.729 5.95 238,119 31,983
057050 현대홈쇼핑 2018.12 0.198 7.13 10,177 1,666
058470 리노공업 2018.12 0.719 14.74 1,504 486
058630 엠게임 2018.12 14.636 21.37 271 25
058850 KTcs 2018.12 0.16 9.38 10,161 114
059090 미코 2018.12 0.213 10.54 2,274 323
060150 인선이엔티 2018.12 0.037 15.01 1,760 155
060250 NHN한국사이버결제 2018.12 0.27 15.51 4,327 168
061040 알에프텍 2018.12 0.247 18.99 2,403 66
061250 화일약품 2018.12 0.205 18.47 1,021 51
061970 엘비세미콘 2018.12 0.128 11.7 2,757 188
063170 서울옥션 2018.12 0.151 20.37 620 71
063570 한국전자금융 2018.12 3.451 14.48 2,944 123
064760 티씨케이 2018.12 0.393 10.07 1,705 469
064820 케이프 2018.12 0.116 19.27 2,546 42
065130 탑엔지니어링 2018.12 0.295 6.74 9,176 325
065510 휴비츠 2018.12 0.105 13.48 789 76
065560 녹원씨엔아이 2018.12 0.018 17.81 764 62
065770 CS 2018.12 0.028 8.6 587 41


C:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'total_revenue' at row 1")
  result = self._query(query)
C:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'bps' at row 1")
  result = self._query(query)
C:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'reserve_ration' at row 1")
  result = self._query(query)
C:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'eps' at row 1")
  result = self._query(query)
C:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'total_income' at row 1")
  result = self._query(query)
C:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'gross_profit' at row 1")
  result = self._query(query)
C:\ProgramData\Anac

066620 국보디자인 2018.12 1.815 7.24 2,543 139
066900 디에이피 2018.12 0.19 8.05 3,041 75
067160 아프리카TV 2018.12 0.514 20.8 1,266 213
067280 멀티캠퍼스 2018.12 0.361 15.06 2,400 176
067920 이글루시큐리티 2018.12 0.056 5.72 647 88
067990 도이치모터스 2018.12 0.005 4.02 10,583 347
068330 일신바이오 2018.12 0.523 25.07 154 27
068790 DMS 2018.12 0.024 6.94 2,987 152
069080 웹젠 2018.12 0.225 14.94 2,189 501
069510 에스텍 2018.12 0.053 5.38 3,064 200
069730 DSR제강 2018.12 0.177 8.38 1,853 74
071050 한국금융지주 2018.12 1.826 6.92 88,267 5,159
071055 한국금융지주우 2018.12 1.072 4.06 88,267 5,159
071200 인피니트헬스케어 2018.12 0.132 29.21 643 48
072710 농심홀딩스 2018.12 0.463 6.94 5,432 468
072990 에이치시티 2018.12 0.004 16.84 364 36
073560 우리손에프앤지 2018.12 1.71 4.03 2,368 274
074430 아미노로직스 2018.12 0.013 18.85 184 80
074600 원익QnC 2018.12 0.162 6.49 2,665 412
075970 동국알앤에스 2018.12 0.96 14.72 1,016 31
077280 한컴지엠디 2018.12 0.396 16.07 110 16
077360 덕산하이메탈 2018.12 0.169 8.44 478 121
077500 유니퀘스트 2018.12 0.643 14.43 2,703 106


C:\ProgramData\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1265, "Data truncated for column 'quick_ratio' at row 1")
  result = self._query(query)


078000 텔코웨어 2018.12 0.986 19.5 421 63
079170 한창산업 2018.12 1.142 20.18 623 14
079370 제우스 2018.12 0.06 3.86 4,536 310
079940 가비아 2018.12 0.211 11.54 1,287 153
079950 인베니아 2018.12 0.029 11.99 1,728 57
080000 에스엔유 2018.12 0.15 9.62 826 98
080220 제주반도체 2018.12 0.068 9.46 1,487 91
080720 한국유니온제약 2018.12 0.191 11.44 547 58
081580 성우전자 2018.12 0.3 5.21 1,586 88
081660 휠라코리아 2018.12 0.178 22.78 29,546 2,100
000070 삼양홀딩스 2018.12 0.116 9.95 25,635 1,025
000075 삼양홀딩스우 2018.12 0.07 6.01 25,635 1,025
000210 대림산업 2018.12 0.193 6.12 109,845 6,781
000215 대림산업우 2018.12 0.073 2.32 109,845 6,781
000270 기아차 2018.12 0.608 11.82 541,698 11,559
000400 롯데손해보험 2018.12 0.174 3.88 35,245 913
000480 조선내화 2018.12 0.022 6.33 7,892 580
000590 CS홀딩스 2018.12 1.831 11.76 1,226 126
000660 SK하이닉스 2018.12 0.062 2.83 404,451 155,400
000720 현대건설 2018.12 0.179 15.95 167,309 5,353
000810 삼성화재 2018.12 7.537 12.71 222,090 10,733
000815 삼성화재우 2018.12 4.861 8.2 222,090 10,733
000850 화천기공 2018.12 0.158 7.14 2,156 127
000860 강남제비스코 

025900 동화기업 2018.12 4.751 11.78 7,600 510
026890 디피씨 2018.12 0.348 21.15 1,577 116
028100 동아지질 2018.12 1.094 10.52 3,607 187
028150 GS홈쇼핑 2018.12 0.291 8.8 11,044 1,339
028260 삼성물산 2018.12 0.07 11.78 311,556 17,482
028670 팬오션 2018.12 2.459 15.6 26,684 1,486
029460 케이씨 2018.12 0.002 0.82 5,261 2,106
029530 신도리코 2018.12 1.246 13.2 5,579 350
029960 코엔텍 2018.12 0.296 15.59 722 245
030000 제일기획 2018.12 9.583 19.95 34,779 1,319
030190 NICE평가정보 2018.12 0.696 17.35 3,838 383
030200 KT 2018.12 0.254 11.3 234,601 7,623
030530 원익홀딩스 2018.12 0.627 3.57 5,002 964
030610 교보증권 2018.12 0.779 4.25 12,501 773
031310 아이즈비전 2018.12 0.071 6.69 1,602 109
031330 에스에이엠티 2018.12 0.636 5.17 11,760 319
031430 신세계인터내셔날 2018.12 0.174 24.16 12,627 577
031820 콤텍시스템 2018.12 0.019 9.7 1,252 66
031980 피에스케이홀딩스 2018.12 0.131 4.67 3,287 535
032280 삼일 2018.12 0.411 12.45 976 26
032580 피델릭스 2018.12 0.017 10.3 754 35
032830 삼성생명 2018.12 0.23 9.81 322,409 17,337
033270 유나이티드제약 2018.12 0.859 11.75 2,119 317
033290 코웰패션 2018.12

243840 신흥에스이씨 2018.12 0.526 24.46 2,154 96
246690 TS인베스트먼트 2018.12 0.089 12.7 113 41
247540 에코프로비엠 2018.12 0.026 3.57 5,892 369
248170 샘표식품 2018.12 0.125 7.07 2,756 180
251370 와이엠티 2018.12 6.774 16.11 727 98
251630 브이원텍 2018.12 0.773 18.4 546 153
255440 야스 2018.12 0.056 5.62 1,845 360
256940 케이피에스 2018.12 0.032 4.93 406 68
258610 이더블유케이 2018.12 0.045 10.49 235 28
258830 세종메디칼 2018.12 5.767 17.74 148 36
263020 디케이앤디 2018.12 0.078 7.58 548 37
263690 디알젬 2018.12 1.145 14.7 566 47
263750 펄어비스 2018.12 0.023 17.94 4,048 1,464
264450 유비쿼스 2018.12 8.483 13.05 953 149
265520 AP시스템 2018.12 1.26 13.14 7,142 252
267270 현대건설기계 2018.12 0.491 9.35 32,339 1,411
267290 경동도시가스 2018.12 0.103 4.59 16,065 342
267980 매일유업 2018.12 1.204 10.46 13,006 583
268280 미원에스씨 2018.12 0.233 9.32 3,604 360
272290 이녹스첨단소재 2018.12 4.846 11.42 2,931 305
277070 린드먼아시아 2018.12 0.837 17.69 81 36
278280 천보 2018.12 0.014 0.72 1,201 226
282690 동아타이어 2018.12 0.009 10.78 1,613 162
285490 노바텍 2018.12 0.257 14.99 259 69
286940 롯데정보통

In [56]:
# from openpyxl import Workbook
# from openpyxl import load_workbook
# wb = load_workbook('stock_data.xlsx',data_only=True)
# ws = wb['Sheet1']
# #iter() 함수를 통해 열의 데이터를 쭉 뽑아낸다. 자바의 이터레이터 같은 기능
# iter_rows = iter(ws.rows)
# next(iter_rows)
# for row in iter_rows:
#     print(row[0].value)
# code_df에 있는 '종목코드' 컬럼을 0을 채운 6자리 포멧으로 맞춰준다.

#code_df.종목코드 = code_df.종목코드.map('{:06d}'.format)

054800
055550
057050
058470
058850
059090
060150
061250
063170
063570
064760
065130
065510
065560
065770
066620
066900
067280
067920
067990
068790
069080
069510
069730
071050
071055
072710
072990
073560
074600
075970
077280
077360
077500
079370
079940
079950
080000
080220
080720
081580
000070
000075
000210
000215
000270
000400
000480
000590
000660
000720
000810
000815
000850
000860
000880
000885
001130
001340
001465
001500
001540
001560
001570
001680
001685
001810
001820
001940
002030
002170
002290
002310
002410
002460
002700
002710
002810
002870
002960
003000
003030
003100
003200
003220
003230
003240
003300
003470
003475
003530
003535
003540
003545
003547
003780
003800
003830
004000
004170
004250
004255
004360
004365
004430
004450
004490
004590
004690
004800
004960
004970
004980
004985
005010
005190
005290
005670
005680
005870
005930
005935
005940
005945
005990
006140
006220
006260
006405
006650
006740
006840
007310
007330
007340
007540
007590
007680
007690
007700
008060
008250
008350

In [33]:

def date_format(d):
    import datetime as dt
    
    d=str(d).replace('-','.')
    yyyy=int(d.split('.')[0])
    mm=int(d.split('.')[1])
    dd=int(d.split('.')[2])
    this_date = dt.date(yyyy,mm,dd)
    return this_date

def historical_index_naver(index_cd, start_date='', end_date='', page_n=1, last_page=0):
    import datetime as dt
    from urllib.request import urlopen
    from bs4 import BeautifulSoup
    
    if start_date:
        start_date = date_format(start_date)
    else:
        start_date = dt.date.today()
    if end_date:
        end_date = date_format(end_date)
    else:
        end_date = dt.date.today()
    
    naver_index = 'https://finance.naver.com/item/sise_day.nhn?code='+index_cd+'&page='+str(page_n)
    
    source = urlopen(naver_index).read()
    source = BeautifulSoup(source, 'lxml')
    
    dates = source.find_all('span',class_='tah p10 gray03')
    prices = source.find_all('td',class_='num')
    for n in range(len(dates)):
        
        if dates[n].text.split('.')[0].isdigit():
            #날짜처리
            this_date = dates[n].text
            this_date = date_format(this_date)
            
            if this_date <= end_date  and this_date >= start_date:
                
                #종가처리
                this_close = prices[n*6].text

                this_close = this_close.replace(',','')
                this_close = float(this_close)

                #딕셔너리에 저장
                historical_prices[this_date] = this_close
            elif this_date < start_date:
                #start_date 이전이면 함수 종료
                return

        #페이지 내비게이션
    if last_page == 0:
        last_page = source.find('td',class_='pgRR').find('a')['href']
        last_page = last_page.split('&')[1]
        last_page = last_page.split('=')[1]
        last_page = int(last_page)
            
        #다음 페이지 호출
    if page_n < last_page:
        page_n = page_n + 1
        historical_index_naver(index_cd, start_date, end_date, page_n, last_page)
    return historical_prices

In [34]:

# 함수호출 예시
index_cd='005930'
historical_prices = dict()
print(historical_prices)
historical_index_naver(index_cd,'2018-12-28','2018-12-28')
a = historical_prices
a

{}


{datetime.date(2018, 12, 28): 38700.0}

In [44]:
def perFunction(code):
    
    index_cd=code
    #historical_prices = dict()
    historical_prices = historical_index_naver(index_cd,'2018-12-28','2018-12-28')
    a = historical_prices
    #per 값을 구하기 위해 연말의 종가값을 구하기 위한 작업. str 
    endPrice = str(a.values())
    startindex = endPrice.find('[')
    endindex = endPrice.rfind('.')
    endPrice = int(endPrice[startindex+1:endindex])
    
    return endPrice

In [36]:
# ep = perFunction('005930')
# print(ep)

dict_values([38700.0]) ================ 18
38700


In [ ]:
#데이터 엑셀화 함수. 매개변수는 엑셀파일로 만들 합산된 데이터프레임
def excelWrite(cpdata):
    import openpyxl
    #엑셀에 값 저장하기 전에 컬럼 이름을 변경시켜준다. db에 저장하기 편하게 하기 위한 작업


    #병합된 데이터를 엑셀문서로 만들어준다. 데이터프레임의 컬럼과 엑셀문서에 생성하는 컬럼명이 일치해야 한다.
    #파일명, 시트명, 컬럼명 확인 잘 하고 실행. 
    cpdata.to_excel('./stockss.xlsx',
                    sheet_name='Sheet1',
                    columns=("cp_code","cp_name","date","peg","per","total_revenue","total_income","gross_profit","quick_ratio","eps","bps","roe",
                             "payout_ratio","debt_ratio","net_profit_margin","yield_ratio",
                             "orperating_margin","reserve_ration","per_share"),
                    header=True,
                    index=0,
                    index_label=None,
                    startrow=0,
                    startcol=0,
                    engine=None,
                    merge_cells=True,
                    encoding="euc_kr",
                    inf_rep='inf',
                    verbose=True,
                    freeze_panes=None)


In [87]:
# def date_format(d):
# 	d=str(d).replace('-','.')
# 	yyyy=int(d.split('.')[0])
# 	mm=int(d.split('.')[1])
# 	dd=int(d.split('.')[2])
# 	this_date = dt.date(yyyy,mm,dd)
# 	return this_date

In [11]:
# import pandas as pd
# from pandas import Series,DataFrame
# import numpy as np

# df2 = DataFrame(np.arange(8).reshape(2,4),
#                 index=[1,0],
#                 columns=['a','b','c','d'])

# df2 = df2.sort_index()
# df2
# df2 = df2.sort_index(ascending=False)
# df2


,a,b,c,d
1,0,1,2,3
0,4,5,6,7
